In [0]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

import logging
import json
import re


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
DISPLAY_LIMIT = 20

Link to the repo: https://github.com/taidopurason/bdm-project-1

### Structure of this notebook
1. Read data from zipped json files into one dataframe.
2. Apply necessary (cleaning) transformations to the dataframe.
3. Create the new DataFrames corresponding to our Warehouse Schema.  
4. Save the DataFrames as Delta tables.
5. Demonstrate adding new entries to the warehouse.
6. Demonstrate queries on the data.

### 1. Extract the data
To check how we downloaded the data from the source, see https://github.com/taidopurason/bdm-project-1/blob/main/Loading%20Data.ipynb. We split the downloaded data into files where each file contains 250,000 json objects.

In [0]:
# Uncomment one or the other line.

# This reads ALL splits into one dataframe
#_df = spark.read.option("multiline", True).json('dbfs:/user/dblpv13/dblpv13.*.json.gz')

# For a faster setup, read just one split
_df = spark.read.option("multiline", True).json('dbfs:/user/dblpv13/dblpv13.0.json.gz')

In [0]:
# Immediately delete the abstract column because they look really annoying on GitHub.
_df = _df.drop(F.col('abstract'))

_df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _id: string (nullable = true)
 |    |    |-- bio: string (nullable = true)
 |    |    |-- email: string (nullable = true)
 |    |    |-- gid: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- name_zh: string (nullable = true)
 |    |    |-- oid: string (nullable = true)
 |    |    |-- oid_zh: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |    |    |-- org: string (nullable = true)
 |    |    |-- org_zh: string (nullable = true)
 |    |    |-- orgid: string (nullable = true)
 |    |    |-- orgs: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- orgs_zh: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- sid: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- fos: array (nullable = 

### 2. Transform the data
TODO: delete all entries where any reference is null.

TODO: delete all entries where any author id is null.

We don't want any null values in the FK columns.

In [0]:
logger.info(f"Initially, there were {_df.count()} rows of data")

INFO:__main__:Initially, there were 250000 rows of data


In [0]:
# Drop entries with 1-word titles or empty authors or nonexistant _id or 
_df = (_df.filter((F.size(F.col('authors')) > 0) & # By default F.size() returns -1 if the value is null.
                  (F.size(F.split(F.col('title'), ' ')) > 1) &  
                  (F.col('_id') != '') & 
                  (F.col('_id').isNotNull()) & 
                  ~(F.array_contains(F.col('references'), ''))))

In [0]:
# remove forewords
_df = _df.filter(~F.lower(F.col("title")).contains("foreword"))

In [0]:
# Convert n_citation data type to int
_df = _df.withColumn('n_citation', F.col('n_citation').cast('int'))

In [0]:
# Replace empty language values with null.
_df = _df.withColumn('lang', F.when(F.col('lang') == '', None).otherwise(F.col('lang')))

In [0]:
# Replace empty 'keyword' and 'fos' arrays with null values.
_df = (_df.withColumn('keywords', F.when(F.size(F.col('keywords')) == 0, None).otherwise(F.col('keywords')))
          .withColumn('fos', F.when(F.size(F.col('fos')) == 0, None).otherwise(F.col('fos'))))

In [0]:
# Replace non-numeric page numbers with nulls and convert column type to int. Then replace 0 page numbers with nulls as well.
_df = (_df.withColumn('page_start', F.when(F.col('page_start').cast('int').isNotNull(), F.col('page_start')).otherwise(None)) # replace non-numeric page numbers with null
          .withColumn('page_end', F.when(F.col('page_end').cast('int').isNotNull(), F.col('page_end')).otherwise(None))
          .withColumn('page_start', F.col('page_start').cast('int')) # convert column type to int
          .withColumn('page_end', F.col('page_end').cast('int'))
          .withColumn('page_start', F.when(F.col('page_start') == 0, None).otherwise(F.col('page_start'))) # replace 0 page numbers with null as well
          .withColumn('page_end', F.when(F.col('page_end') == 0, None).otherwise(F.col('page_end'))))

In [0]:
# Replace empty dois with nulls.
_df = _df.withColumn('doi', F.when(F.col('doi') == '', None).otherwise(F.col('doi')))

In [0]:
# Replace empty years with nulls and change data type to int.
_df = (_df.withColumn('year', F.when(F.col('year') == 0, None).otherwise(F.col('year')))
          .withColumn('year', F.col('year').cast('int')))

In [0]:
# Replace non-numeric volume and issue numbers with null and convert data types to int. Then repalce 0 values with null as well.
_df = (_df.withColumn('volume', F.when(F.col('volume').cast('int').isNotNull(), F.col('volume')).otherwise(None)) # replace non-numeric values
          .withColumn('issue', F.when(F.col('issue').cast('int').isNotNull(), F.col('issue')).otherwise(None))
          .withColumn('volume', F.col('volume').cast('int')) # convert column type to int
          .withColumn('issue', F.col('issue').cast('int'))
          .withColumn('volume', F.when(F.col('volume') == 0, None).otherwise(F.col('volume'))) # replace 0 issue and volume numbers with null as well.
          .withColumn('issue', F.when(F.col('issue') == 0, None).otherwise(F.col('issue'))))

In [0]:
# replace empty strings in some columns with nulls
def replace_empty_string(col):
    return F.when(col == "", None).otherwise(col)

venue = F.col("venue")
for col in ["_id", "issn", "name", "name_d", "name_s", "online_issn", "publisher", "raw", "raw_zh", "t"]:
    venue = venue.withField(col, replace_empty_string(F.col(f"venue.{col}")))
    
_df = (
    _df
    .withColumn("venue", venue)
    .withColumn("issn", replace_empty_string(F.col("issn")))
    .withColumn("isbn", replace_empty_string(F.col("isbn")))
    .withColumn("isbn", F.when(F.col("isbn") == "isbn", None).otherwise(F.col("isbn")))
    .withColumn("issn", F.when(F.col("issn") == "issn", None).otherwise(F.col("issn")))
)

In [0]:
# fix incorrect issn
_df = (_df
           .withColumn("issn",
                       F.when(F.length(F.col("issn")) == 9, F.col("issn"))
                       .when(F.length(F.col("issn")) == 8, F.concat(F.col("issn").substr(1, 4), F.lit("-"), F.col("issn").substr(5, 4)))
                       .when(F.col("issn").contains("E-ISBN"), F.col("issn").substr(1, 9))
                       .otherwise(None)
                      )
           .withColumn("venue", 
                       F.col("venue")
                       .withField("issn", F.coalesce(F.col("venue.issn"), F.col("issn")))
                       )
           .drop("issn")
          )

In [0]:
# replace venue with null fields with null
venue_is_empty = (
    F.col("venue.issn").isNull() &
    F.col("venue.name").isNull() &
    F.col("venue.name_d").isNull() &
    F.col("venue.name_s").isNull() &
    F.col("venue.online_issn").isNull() &
    F.col("venue.publisher").isNull() &
    F.col("venue.raw").isNull() &
    F.col("venue.raw_zh").isNull()
)
_df = _df.withColumn("venue", F.when(venue_is_empty, None).otherwise(F.col("venue")))

In [0]:
# remove rows with null venues
_df = _df.filter(F.col("venue").isNotNull())

In [0]:
# coalescing venue._id and venue.issn to make up for missing ids
_df = _df.withColumn("venue", F.col("venue").withField("_id", F.coalesce(F.col("venue._id"), F.col("venue.issn"))))

# removing rows with venue id null
_df = _df.filter(F.col("venue._id").isNotNull())

In [0]:
logger.info(f"Now, there are {_df.count()} rows of data")

_df.printSchema()

INFO:__main__:Now, there are 163401 rows of data
root
 |-- _id: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _id: string (nullable = true)
 |    |    |-- bio: string (nullable = true)
 |    |    |-- email: string (nullable = true)
 |    |    |-- gid: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- name_zh: string (nullable = true)
 |    |    |-- oid: string (nullable = true)
 |    |    |-- oid_zh: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |    |    |-- org: string (nullable = true)
 |    |    |-- org_zh: string (nullable = true)
 |    |    |-- orgid: string (nullable = true)
 |    |    |-- orgs: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- orgs_zh: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- sid: string (nullable = true)
 |-- doi: stri

In [0]:
# Display endresult
display(_df.limit(DISPLAY_LIMIT))

_id,authors,doi,fos,isbn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,venue,volume,year
53e99784b7602d9701f3e151,"List(List(53f46797dabfaeb22f542630, null, null, null, Jairo Rocha, null, null, null, null, null, null, null, null, null, null), List(54328883dabfaeb4c6a8a699, null, null, null, Theo Pavlidis, null, null, null, null, null, null, null, null, null, null))",10.1109/ICDAR.1993.395663,"List(Intelligent character recognition, Pattern recognition, Computer science, Feature (computer vision), Document processing, Handwriting recognition, Optical character recognition, Feature extraction, Feature (machine learning), Artificial intelligence, Intelligent word recognition)",null,null,"List(handwriting recognition, prototypes, image segmentation, computer science, expert systems, knowledge base, pattern recognition, usability, optical character recognition, shape, feature extraction)",en,17,605,602,null,"List(53e99cf5b7602d97025ace63, 557e8a7a6fee0fe990caa63d, 53e9a96cb7602d97032c459a, 53e9b929b7602d9704515791, 557e59ebf6678c77ea222447)",A solution to the problem of touching and broken characters.,List(http://dx.doi.org/10.1109/ICDAR.1993.395663),"List(53a72a4920f7420be8bfa51b, null, null, International Conference on Document Analysis and Recognition, null, null, null, ICDAR-1, null, null, null, null, 0)",null,1993
53e99784b7602d9701f3e15d,"List(List(53f43b03dabfaedce555bf2a, null, null, null, Min Pan, null, null, null, null, null, null, null, null, null, null), List(53f45ee9dabfaee43ecda842, null, null, null, Chris C. N. Chu, null, null, null, null, null, null, null, null, null, null), List(53f42e8cdabfaee1c0a4274e, null, null, null, Hai Zhou, null, null, null, null, null, null, null, null, null, null))",10.1109/ISCAS.2005.1465124,"List(Delay calculation, Timing failure, Monte Carlo method, Sequential logic, Statistical static timing analysis, Shortest path problem, Computer science, Algorithm, Clock skew, Static timing analysis, Statistics)",0-7803-8834-8,null,"List(sequential circuits, statistical distributions, set-up time constraints, register-to-register paths, statistical static timing analysis, integrated circuit modelling, parameter estimation, statistical analysis, circuit model, path delays, deep sub-micron technology, timing, delay distributions, delays, circuit timing, shortest path variations, hold time constraints, integrated circuit yield, process variations, integrated circuit layout, high-performance circuit designs, clock skew, timing yield estimation, deterministic static timing analysis, monte carlo simulation, design method, static timing analysis, design methodology, process variation, shortest path, registers, circuit design, circuit analysis)",en,28,null,2461,//static.aminer.org/pdf/PDF/000/423/329/timing_yield_estimation_using_statistical_static_timing_analysis.pdf,"List(53e9a8a9b7602d97031f6bb9, 599c7b6b601a182cd27360da, 53e9b443b7602d9703f3e52b, 53e9a6a6b7602d9702fdc57e, 599c7b6a601a182cd2735703, 53e9aad9b7602d970345afea, 5582821f0cf2bf7bae57ac18, 5e8911859fced0a24bb9a2ba, 53e9b002b7602d9703a5c932)",Timing yield estimation using statistical static timing analysis,"List(http://dx.doi.org/10.1109/ISCAS.2005.1465124, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=1465124)","List(53a72e2020f7420be8c80142, null, null, International Symposium on Circuits and Systems, null, null, null, ISCAS (3), null, null, null, null, 0)",null,2005
53e99784b7602d9701f3f411,"List(List(548a2e3ddabfae9b40134fbc, null, null, null, Harry M. Sneed, null, null, null, null, null, null, null, null, null, null))",10.1109/CMPSAC.2002.1044548,"List(XML Base, World Wide Web, XML framework, XML Encryption, Efficient XML Interchange, SGML, Programming language, Software engineering, XML, XML validation, Computer science, cXML)",0-7695-1727-7,null,"List(Internet, hypermedia markup languages, information resources, systems re-engineering, COBOL, PL/I, World Wide Web, XML, batch programs, data

### 3. Create the new DFs
#### 3.1. Venue DF

In [0]:
# creating the venues df
venues_df = (_df
             .withColumn("has_volume_or_issue", F.when(F.col("volume").isNotNull() | F.col("issue").isNotNull(), True).otherwise(None))
             .select("venue.*", "has_volume_or_issue")
             .filter(F.col("_id").isNotNull())
             .drop("src", "sid", "type").distinct())

# removing the columns from the original df
_df = _df.withColumn("venue_id", F.col("venue._id")).drop("venue")

display(venues_df.limit(DISPLAY_LIMIT))

_id,issn,name,name_d,name_s,online_issn,publisher,raw,raw_zh,t,has_volume_or_issue
53a72bad20f7420be8c2d5af,null,null,IEEE International Conference on Cognitive Informatics,null,null,null,IEEE ICCI,null,null,null
555036db7cea80f9541603d7,null,null,null,null,null,null,J. Multivariate Analysis,null,null,true
555036ba7cea80f95414db52,null,null,Advances in Operations Research,null,null,null,Adv. Operations Research,null,null,true
53a72e2020f7420be8c80142,null,null,International Symposium on Circuits and Systems,null,null,null,ISCAS (3),null,null,null
53a724b320f7420be8b37f4c,null,null,Hawaii International Conference on System Sciences,null,null,null,HICSS,null,null,null
53a72cf620f7420be8c548e2,null,null,null,null,null,null,MobiSys,null,null,null
555036c77cea80f954155203,null,null,IEEE Transactions on Circuits and Systems,null,null,null,IEEE Trans. on Circuits and Systems,null,null,true
0377-2217,0377-2217,null,null,null,null,North-Holland,European Journal of Operational Research,null,J,true
555036b77cea80f95414b7de,1869-1919,null,null,null,null,null,SCIENCE CHINA Information Sciences,null,null,true
53a72a4920f7420be8bfa51b,null,null,International Conference on Document Analysis and Recognition,null,null,null,ICDAR-1,null,null,null


In [0]:
# combining rows with the same id, but different column values
# taking the first non-null value for the id as the column value

venue_columns = (
    "issn",
    "name",
    "name_d",
    "name_s",
    "raw",
    "raw_zh",
    "online_issn",
    "publisher",
    "t",
    "has_volume_or_issue"
)

venues_df = venues_df.groupBy(F.col("_id")).agg(*(F.first(F.col(col), ignorenulls=True).alias(col) for col in venue_columns))

In [0]:
venues_df = (
    venues_df
    # coalescing the name and raw columns
    .withColumn("raw", F.coalesce(
            F.col("raw"), 
            F.col("raw_zh"),
        ))
    .withColumn("name", F.coalesce(
            F.col("name"), 
            F.col("name_d"),
        ))
    .drop("name_d", "name_s", "raw_zh") 
    # creating the type field
    .withColumn("type",            
               F.when(
                   (
                       F.col("raw").contains("@") | 
                       F.lower(F.col("raw")).contains("workshop") |
                       F.lower(F.col("name")).contains("workshop")
                   ), 
                   "Workshop"
               ).when(
                   (F.col("t") == "J"),
                   "Journal"
               ).when(
                   (
                       (F.col("t") == "C") |
                       F.lower(F.col("raw")).contains("conference") |
                       F.lower(("name")).contains("conference") |
                       F.lower(F.col("raw")).contains("symposium") |
                       F.lower(("name")).contains("symposium") |
                       F.lower(F.col("raw")).contains("proceedings") |
                       F.lower(("name")).contains("proceedings")
                   ),
                   "Conference"
               ).when(
                   (
                       F.lower(F.col("raw")).contains("journal") |
                       F.lower(("name")).contains("journal") |
                       F.col("has_volume_or_issue")
                   ),
                   "Journal"
               ).otherwise(None)
      )
    .drop("t", "has_volume_or_issue")
)

display(venues_df.limit(DISPLAY_LIMIT))

_id,issn,name,raw,online_issn,publisher,type
0001-0782,0001-0782,null,COMMUNICATIONS OF THE ACM,1557-7317,null,Journal
0001-253X,0001-253X,null,ASLIB PROCEEDINGS,1758-3748,null,Journal
0001-2815,0001-2815,null,TISSUE ANTIGENS,null,null,Journal
0001-4966,0001-4966,null,JOURNAL OF THE ACOUSTICAL SOCIETY OF AMERICA,1520-8524,null,Journal
0001-5903,0001-5903,null,Acta Informatica,null,null,Journal
0001-8708,0001-8708,null,Advances in Mathematics,null,Academic Press,Journal
0002-8231,0002-8231,null,JOURNAL OF THE AMERICAN SOCIETY FOR INFORMATION SCIENCE,null,null,Journal
0002-9149,0002-9149,null,AMERICAN JOURNAL OF CARDIOLOGY,null,null,Journal
0002-9343,0002-9343,null,AMERICAN JOURNAL OF MEDICINE,null,null,Journal
0002-9378,0002-9378,null,AMERICAN JOURNAL OF OBSTETRICS AND GYNECOLOGY,null,null,Journal


In [0]:
display(_df.limit(DISPLAY_LIMIT))

_id,abstract,authors,doi,fos,isbn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,volume,year,venue_id
53e99784b7602d9701f3e151,,"List(List(53f46797dabfaeb22f542630, null, null, null, Jairo Rocha, null, null, null, null, null, null, null, null, null, null), List(54328883dabfaeb4c6a8a699, null, null, null, Theo Pavlidis, null, null, null, null, null, null, null, null, null, null))",10.1109/ICDAR.1993.395663,"List(Intelligent character recognition, Pattern recognition, Computer science, Feature (computer vision), Document processing, Handwriting recognition, Optical character recognition, Feature extraction, Feature (machine learning), Artificial intelligence, Intelligent word recognition)",null,null,"List(handwriting recognition, prototypes, image segmentation, computer science, expert systems, knowledge base, pattern recognition, usability, optical character recognition, shape, feature extraction)",en,17,605,602,null,"List(53e99cf5b7602d97025ace63, 557e8a7a6fee0fe990caa63d, 53e9a96cb7602d97032c459a, 53e9b929b7602d9704515791, 557e59ebf6678c77ea222447)",A solution to the problem of touching and broken characters.,List(http://dx.doi.org/10.1109/ICDAR.1993.395663),null,1993,53a72a4920f7420be8bfa51b
53e99784b7602d9701f3e15d,"As process variations become a significant problem in deep sub-micron technology, a shift from deterministic static timing analysis to statistical static timing analysis for high-performance circuit designs could reduce the excessive conservatism that is built into current timing design methods. We address the timing yield problem for sequential circuits and propose a statistical approach to handle it. We consider the spatial and path reconvergence correlations between path delays, set-up time and hold time constraints, and clock skew due to process variations. We propose a method to get the timing yield based on the delay distributions of register-to-register paths in the circuit On average, the timing yield results obtained by our approach have average errors of less than 1.0% in comparison with Monte Carlo simulation. Experimental results show that shortest path variations and clock skew due to process variations have considerable impact on circuit timing, which could bias the timing yield results. In addition, the correlation between longest and shortest path delays is not significant.","List(List(53f43b03dabfaedce555bf2a, null, null, null, Min Pan, null, null, null, null, null, null, null, null, null, null), List(53f45ee9dabfaee43ecda842, null, null, null, Chris C. N. Chu, null, null, null, null, null, null, null, null, null, null), List(53f42e8cdabfaee1c0a4274e, null, null, null, Hai Zhou, null, null, null, null, null, null, null, null, null, null))",10.1109/ISCAS.2005.1465124,"List(Delay calculation, Timing failure, Monte Carlo method, Sequential logic, Statistical static timing analysis, Shortest path problem, Computer science, Algorithm, Clock skew, Static timing analysis, Statistics)",0-7803-8834-8,null,"List(sequential circuits, statistical distributions, set-up time constraints, register-to-register paths, statistical static timing analysis, integrated circuit modelling, parameter estimation, statistical analysis, circuit model, path delays, deep sub-micron technology, timing, delay distributions, delays, circuit timing, shortest path variations, hold time constraints, integrated circuit yield, process variations, integrated circuit layout, high-performance circuit designs, clock skew, timing yield estimation, deterministic static timing analysis, monte carlo simulation, design method, static timing analysis, design methodology, process variation, shortest path, registers, circuit design, circuit analysis)",en,28,null,2461,//static.aminer.org/pdf/PDF/000/423/329/timing_yield_estimation_using_statistical_static_timing_analysis.pdf,"List(53e9a8a9b7602d97031f6bb9, 599c7b6b601a182cd27360da, 53e9b443b7602d9703f3e52b, 53e9a6a6b7602d9702fdc57e, 599c7b6a601a182cd2735703, 53e9aad9

#### 3.2. Author DF

In [0]:
# Create the Authors DF

df2 = _df.withColumn('auth_expl', F.explode(F.col("authors"))) # explode the authors array
df2 = (df2.withColumn('auth_id', F.col('auth_expl._id')) # separate the authors id and name
          .withColumn('auth_name', F.col('auth_expl.name')))

authors_df = df2.select('auth_id', 'auth_name').distinct() # make the authors df of distinct auth_id and auth_name pairs
# there were only 92 rows where both auth id and name were null.
# altogether, there are 400k unique authors.

authors_df = (authors_df.withColumnRenamed('auth_id', 'ID')
                        .withColumnRenamed('auth_name', 'Name'))

del df2
display(authors_df.limit(DISPLAY_LIMIT))

ID,Name
53f46a22dabfaee0d9c3d5e5,Shuguo Yang
53f45ee9dabfaee43ecda842,Chris C. N. Chu
548a2e3ddabfae9b40134fbc,Harry M. Sneed
53f45e2adabfaeb22f51d645,Luís Macedo
53f42e8cdabfaee1c0a4274e,Hai Zhou
53f43b03dabfaedce555bf2a,Min Pan
54328883dabfaeb4c6a8a699,Theo Pavlidis
53f4775edabfaee4dc891b69,Kenji Sugawara
53f4333fdabfaeb22f451979,Norman Wilde
53f46e66dabfaee02adb48fd,Shengqi Ye


In [0]:
# Generate the authors FK column in the original DF
_df = _df.withColumn('Author_ID', F.col('authors._id'))
display(_df.limit(DISPLAY_LIMIT))

_id,abstract,authors,doi,fos,isbn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,volume,year,venue_id,Author_ID
53e99784b7602d9701f3e151,,"List(List(53f46797dabfaeb22f542630, null, null, null, Jairo Rocha, null, null, null, null, null, null, null, null, null, null), List(54328883dabfaeb4c6a8a699, null, null, null, Theo Pavlidis, null, null, null, null, null, null, null, null, null, null))",10.1109/ICDAR.1993.395663,"List(Intelligent character recognition, Pattern recognition, Computer science, Feature (computer vision), Document processing, Handwriting recognition, Optical character recognition, Feature extraction, Feature (machine learning), Artificial intelligence, Intelligent word recognition)",null,null,"List(handwriting recognition, prototypes, image segmentation, computer science, expert systems, knowledge base, pattern recognition, usability, optical character recognition, shape, feature extraction)",en,17,605,602,null,"List(53e99cf5b7602d97025ace63, 557e8a7a6fee0fe990caa63d, 53e9a96cb7602d97032c459a, 53e9b929b7602d9704515791, 557e59ebf6678c77ea222447)",A solution to the problem of touching and broken characters.,List(http://dx.doi.org/10.1109/ICDAR.1993.395663),null,1993,53a72a4920f7420be8bfa51b,"List(53f46797dabfaeb22f542630, 54328883dabfaeb4c6a8a699)"
53e99784b7602d9701f3e15d,"As process variations become a significant problem in deep sub-micron technology, a shift from deterministic static timing analysis to statistical static timing analysis for high-performance circuit designs could reduce the excessive conservatism that is built into current timing design methods. We address the timing yield problem for sequential circuits and propose a statistical approach to handle it. We consider the spatial and path reconvergence correlations between path delays, set-up time and hold time constraints, and clock skew due to process variations. We propose a method to get the timing yield based on the delay distributions of register-to-register paths in the circuit On average, the timing yield results obtained by our approach have average errors of less than 1.0% in comparison with Monte Carlo simulation. Experimental results show that shortest path variations and clock skew due to process variations have considerable impact on circuit timing, which could bias the timing yield results. In addition, the correlation between longest and shortest path delays is not significant.","List(List(53f43b03dabfaedce555bf2a, null, null, null, Min Pan, null, null, null, null, null, null, null, null, null, null), List(53f45ee9dabfaee43ecda842, null, null, null, Chris C. N. Chu, null, null, null, null, null, null, null, null, null, null), List(53f42e8cdabfaee1c0a4274e, null, null, null, Hai Zhou, null, null, null, null, null, null, null, null, null, null))",10.1109/ISCAS.2005.1465124,"List(Delay calculation, Timing failure, Monte Carlo method, Sequential logic, Statistical static timing analysis, Shortest path problem, Computer science, Algorithm, Clock skew, Static timing analysis, Statistics)",0-7803-8834-8,null,"List(sequential circuits, statistical distributions, set-up time constraints, register-to-register paths, statistical static timing analysis, integrated circuit modelling, parameter estimation, statistical analysis, circuit model, path delays, deep sub-micron technology, timing, delay distributions, delays, circuit timing, shortest path variations, hold time constraints, integrated circuit yield, process variations, integrated circuit layout, high-performance circuit designs, clock skew, timing yield estimation, deterministic static timing analysis, monte carlo simulation, design method, static timing analysis, design methodology, process variation, shortest path, registers, circuit design, circuit analysis)",en,28,null,2461,//static.aminer.org/pdf/PDF/000/423/329/timing_yield_estimation_using_statistical_static_timing_analysis.pdf,"List(53e9a8a9b7602d97031f6bb9, 599c7b6b601a182cd27360da, 53e9b443b7602d970

#### 3.3. Organization DF

TODO: Make sure that in \_df there is the column 'Org' with the organization id.

In [0]:
# Create the new df

# finds the country names in a list of strings
# modified to only use the first element of the list
# uses regex to remove punctuation from the string and to match the given names of the countries and some abbreviations

def getCountry(s):
    if s is None:
        return None
    arr = []
    countries = ["Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua & Deps", "Argentina", "Armenia", "Australia", "Austria", "Azerbaijan", "Bahamas", "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium", "Belize", "Benin", "Bhutan", "Bolivia", "Bosnia", "Botswana", "Brazil", "Brunei", "Bulgaria", "Burkina", "Burundi", "Cambodia", "Cameroon", "Canada", "Cape Verde", "Central African Republic", "Chad", "Chile", "China", "Colombia", "Comoros", "Congo", "Congo Democratic Republic", "Costa Rica", "Croatia", "Cuba", "Cyprus", "Czech Republic", "Denmark", "Djibouti", "Dominica", "Dominican Republic", "East Timor", "Ecuador", "Egypt", "El Salvador", "Equatorial Guinea", "Eritrea", "Estonia", "Ethiopia", "Fiji", "Finland", "France", "Gabon", "Gambia", "Georgia", "Germany", "Ghana", "Greece", "Grenada", "Guatemala", "Guinea", "Guinea-bissau", "Guyana", "Haiti", "Honduras", "Hungary", "Iceland", "India", "Indonesia", "Iran", "Iraq", "Ireland", "Israel", "Italy", "Ivory Coast", "Jamaica", "Japan", "Jordan", "Kazakhstan", "Kenya", "Kiribati", "South Korea", "Kosovo", "Kuwait", "Kyrgyzstan", "Laos", "Latvia", "Lebanon", "Lesotho", "Liberia", "Libya", "Liechtenstein", "Lithuania", "Luxembourg", "Macedonia", "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali", "Malta", "Marshall Islands", "Mauritania", "Mauritius", "Mexico", "Micronesia", "Moldova", "Monaco", "Mongolia", "Montenegro", "Morocco", "Mozambique", "Myanmar", "Burma", "Namibia", "Nauru", "Nepal", "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria", "Norway", "Romania", "Pakistan", "Palau", "Panama", "Papua New Guinea", "Paraguay", "Peru", "Philippines", "Poland", "Portugal", "Qatar", "Oman", "Russia", "Rwanda", "St Kitts & Nevis", "St Lucia", "Saint Vincent & The Grenadines", "Samoa", "San Marino", "Sao Tome & Principe", "Saudi Arabia", "Senegal", "Serbia", "Seychelles", "Sierra Leone", "Singapore", "Slovakia", "Slovenia", "Solomon Islands", "Somalia", "South Africa", "South Sudan", "Spain", "Sri Lanka", "Sudan", "Suriname", "Swaziland", "Sweden", "Switzerland", "Syria", "Taiwan", "Tajikistan", "Tanzania", "Thailand", "Togo", "Tonga", "Trinidad & Tobago", "Tunisia", "Turkey", "Turkmenistan", "Tuvalu", "Uganda", "Ukraine", "United Arab Emirates", "United Kingdom", "United States", "Uruguay", "Uzbekistan", "Vanuatu", "Vatican City", "Venezuela", "Vietnam", "Yemen", "Zambia", "Zimbabwe"]
    state_names = ["alaska", "alabama", "arkansas", "american samoa", "arizona", "california", "colorado", "connecticut", "district ", "of columbia", "delaware", "florida", "georgia", "guam", "hawaii", "iowa", "idaho", "illinois", "indiana", "kansas", "kentucky", "louisiana", "massachusetts", "maryland", "maine", "michigan", "minnesota", "missouri", "mississippi", "montana", "north carolina", "north dakota", "nebraska", "new hampshire", "new jersey", "new mexico", "nevada", "new york", "ohio", "oklahoma", "oregon", "pennsylvania", "puerto rico", "rhode island", "south carolina", "south dakota", "tennessee", "texas", "utah", "virginia", "virgin islands", "vermont", "washington", "wisconsin", "west virginia", "wyoming"]
    states = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
    
    for i in s:
        if i["org"] is None:
            arr.append(None)
            break
        sent = re.sub("[^a-zA-Z -]", "", i["org"])
        x = None
        for j in countries:
            x = re.search(j.lower(), sent.lower())
            if x is not None:
                if j.lower() == 'india':
                    x = re.search('indiana', sent.lower())
                    if x is not None:
                        arr.append("United States")
                elif j.lower() == 'georgia':
                    x = re.search('USA', sent)
                    if x is not None:
                        arr.append("United States")
                else:
                    arr.append(j)
                break
        if x is None:
            x = re.search("USA", sent)
            if x is not None:
                arr.append("United States")
                break
        if x is None:
            x = re.search("UK", sent)
            if x is not None:
                arr.append("United Kingdom")
                break
        if x is None:
            x = re.search("england", sent.lower())
            if x is not None:
                arr.append("United Kingdom")
                break
        if x is None:
            x = re.search("scotland", sent.lower())
            if x is not None:
                arr.append("United Kingdom")
                break
        if x is None:
            x = re.search("wales", sent.lower())
            if x is not None:
                arr.append("United Kingdom")
                break
        if x is None:
            for j in states:
                x = re.search(j, sent)
                if x is not None:
                    arr.append("United States")
                    break
        if x is None:
            for j in state_names:
                x = re.search(j, sent.lower())
                if x is not None:
                    arr.append("United States")
                    break
        break
                    
    if len(arr) > 0:
        return arr[0]
    else:
        return None

getCountryUDF = udf(getCountry)

In [0]:
# Organization (affiliation of the first author)
# ID - authors.orgid
# Name - authors.org
# Country - getCountryUDF(F.arrays_zip("authors.org"))
def organization(df):
    new_df = df.select(F.col("authors.orgid").getItem(0).alias("ID"),
                       F.col("authors.org").getItem(0).alias("Name"),
                       (getCountryUDF(F.arrays_zip("authors.org"))).alias("Country"))
    new_df = new_df.na.drop("all")
    return new_df

org_df = organization(_df)

In [0]:
# create the new df with new IDs
new_df = org_df.select('ID', 'Name', 'Country').distinct()
new_df = new_df.withColumn('Org', F.monotonically_increasing_id())
new_df = new_df.withColumn('Org', new_df.Org.cast(T.StringType()))
new_df = new_df.fillna('missing_org_id', 'ID')

In [0]:
# add separate org id and name columns to the original df
df2 = _df.withColumn("ID", F.col("authors.orgid").getItem(0)).withColumn("Name", F.col("authors.org").getItem(0))
df2 = df2.fillna('missing_org_id', 'ID')

In [0]:
# Join ID to original dataframe (_df)

# join newly created IDs to the original df and replace missing org ids with generated IDs
_df = df2.join(new_df.select('Org', 'ID', 'Name'), on=['ID', 'Name'])
_df = _df.replace('missing_org_id', None)
_df = _df.withColumn('Org', F.coalesce(_df.ID, _df.Org))
_df = _df.drop('ID', 'Name')

display(_df.limit(DISPLAY_LIMIT))

_id,abstract,authors,doi,fos,isbn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,volume,year,venue_id,Author_ID,Org
53e997d7b7602d9701fd0004,"An ad hoc team setting is one in which teammates must work together to obtain a common goal, but without any prior agreement regarding how to work together. In this work we introduce a role-based approach for ad hoc teamwork, in which each teammate is inferred to be following a specialized role that accomplishes a specific task or exhibits a particular behavior. In such cases, the role an ad hoc agent should select depends both on its own capabilities and on the roles currently selected by other team members. We present methods for evaluating the influence of the ad hoc agent's role selection on the team's utility and we examine empirically how to choose the best suited method for role assignment in a complex environment. Finally, we show that an appropriate assignment method can be determined from a limited amount of data and used successfully in new tasks that the team has not encountered before.","List(List(53f43819dabfaee02acd9ff6, null, null, 5b86b6bfe1cd8e14a34c9598, Katie Genter, null, null, null, null, The University of Texas at Austin, null, 5f71b2841c455f439fe3c6bf, null, null, null), List(53f434b5dabfaeecd694fcdf, null, null, 5b86b6bfe1cd8e14a34c9598, Noa Agmon, null, null, null, null, The University of Texas at Austin, null, 5f71b2841c455f439fe3c6bf, null, null, null), List(53f42d28dabfaeb2acfe712a, null, null, 5b86b6bfe1cd8e14a34c9598, Peter Stone, null, null, null, null, The University of Texas at Austin, null, 5f71b2841c455f439fe3c6bf, null, null, null))",null,"List(Teamwork, Computer science, Knowledge management, Human–computer interaction)",0-9817381-3-3,null,"List(role assignment, appropriate assignment method, specialized role, common goal, role selection, complex environment, present method, team member, team setting, limited amount)",en,1,1252,1251,https://static.aminer.cn/upload/pdf/program/53e997d7b7602d9701fd0004_0.pdf,"List(53e9a309b7602d9702c14b8b, 53e99858b7602d970208f24f, 53e9aa61b7602d97033d4b67, 53e9a1e1b7602d9702ae0e7e, 53e9a6cab7602d9702fff64d)",Role selection in ad hoc teamwork,List(http://dl.acm.org/citation.cfm?id=2343948),null,2012,555037187cea80f954172d7a,"List(53f43819dabfaee02acd9ff6, 53f434b5dabfaeecd694fcdf, 53f42d28dabfaeb2acfe712a)",5f71b2841c455f439fe3c6bf
53e997ddb7602d9701fd2cce,"A social e-book provides not only the original text but also other readers&' comments, and it enables social interactions inside the book. We posited that a social e-book could be a useful tool for collaborative learning, and it could provide new opportunities for classic humanities texts. The research objective is to find the tendencies of reader generated annotations during two social reading projects. For theoretical background, ""the significance of the text - social interaction model"" was used for the analysis conducted in this study, and we classified user generated annotations into three different types. As a result, participants had a tendency to make more annotations about their understanding and appreciation than regarding text interpretation. In addition, the result shows that the social e-book can promote fine-grained interactions. Regarding the comparison of the genres of the contents, the group of people who read the classic and humanities genre is more active than those who read the popular literature genre. For future study, more specific ways to improve interest and understanding will be examined for effective collaborative reading experiences through the social e-book.","List(List(53f466afdabfaeee22a54847, null, null, 5b868af8e1cd8e14a32924f3, Seyeon Lee, null, null, null, null, Graduate School of Culture Technology, KAIST, Yuseong-gu, Daejeon, South Korea, null, 5f71b2831c455f439fe3c61e, null, null, null), List(53f467fadabfaedd74e6ffbd, null, null, 5b868af8e1cd8e14a32924f3, Jea In Kim, null, null, null, null, Graduate

In [0]:
# replace missing org ids in the new df with generated IDs
new_df = new_df.replace('missing_org_id', None)
new_df = new_df.withColumn('ID', F.coalesce(new_df.ID, new_df.Org))
new_df = new_df.drop('Org')

display(new_df.limit(DISPLAY_LIMIT))

ID,Name,Country
5f71b2971c455f439fe3cecb,"Department of Industrial and Systems Engineering, National University of Singapore, Faculty of Engineering, 10 Kent Ridge Crescent, Singapore 119260",Singapore
5f71b2971c455f439fe3cede,"Australian e-Health Research Centre, CSIRO ICT Centre, Australia",Australia
5f71b3711c455f439fe42e84,"Instituto de Física Aplicada, Consejo Superior de Investigaciones Científicas, Serrano 144, 28006 Madrid, Spain",Spain
5f71b2f61c455f439fe3f823,"Department of Electrical Engineering, Biomedical Engineering Laboratory (GPEB), Federal University of Santa Catarina, Florianópolis 88040-900, Brazil",Brazil
5f71b28e1c455f439fe3cad9,"Northwestern Univ., Evanston, IL",United States
5f71b3081c455f439fe3ff73,"Iran Univ Sci & Technol, Sch Comp Engn, Tehran, Iran",Iran
5f71b2ac1c455f439fe3d6cc,"Computer Information Systems Department, Georgia State University, Atlanta, GA 30302-4015, USA",United States
5f71b2841c455f439fe3c6b7,"Computer Science Division, 387 Soda Hall, University of California, Berkeley, CA 94720. E-mail: nir@cs.berkeley.edu",United States
5f71b2b81c455f439fe3dc45,"CACI Products Company, 3333 North Torrey Pines Court, La Jolla, CA",United States
5f71b2831c455f439fe3c634,"Microsoft Research Cambridge, Cambridge, UK",United Kingdom


#### 3.4. DBLP fact table

In [0]:
dblp_df = _df.select('_id','venue_id','Org','Author_ID','references','keywords','fos','title','n_citation','lang','page_start','page_end','doi','isbn','year','volume','issue')

dblp_df = dblp_df.toDF('ID','Venue','Org','Authors','References','Keywords','FOS','Title','NoCitations','Lang','PageStart','PageEnd','DOI','ISBN','Year','Volume','Issue')

In [0]:
display(dblp_df.limit(DISPLAY_LIMIT))

ID,Venue,Org,Authors,References,Keywords,FOS,Title,NoCitations,Lang,PageStart,PageEnd,DOI,ISBN,Year,Volume,Issue
53e997d7b7602d9701fd0004,555037187cea80f954172d7a,5f71b2841c455f439fe3c6bf,"List(53f43819dabfaee02acd9ff6, 53f434b5dabfaeecd694fcdf, 53f42d28dabfaeb2acfe712a)","List(53e9a309b7602d9702c14b8b, 53e99858b7602d970208f24f, 53e9aa61b7602d97033d4b67, 53e9a1e1b7602d9702ae0e7e, 53e9a6cab7602d9702fff64d)","List(role assignment, appropriate assignment method, specialized role, common goal, role selection, complex environment, present method, team member, team setting, limited amount)","List(Teamwork, Computer science, Knowledge management, Human–computer interaction)",Role selection in ad hoc teamwork,1,en,1251,1252,null,0-9817381-3-3,2012,null,null
53e997ddb7602d9701fd2cce,53a7261520f7420be8b66689,5f71b2831c455f439fe3c61e,"List(53f466afdabfaeee22a54847, 53f467fadabfaedd74e6ffbd, 53f43a87dabfaee02acf230a)","List(53e9b86db7602d9704439294, 53e9b130b7602d9703bb0b59, 53e9b39db7602d9703e80f29)","List(social reading project, reader experience, original text, collaborative learning, text interpretation, social interaction model, social e-book, effective collaborative reading experience, social interaction, future study, classic humanities text)","List(Social group, Social relation, World Wide Web, Social media, Collaborative learning, Future study, Psychology, Social learning, Epistemology, Social competence)",Assessing the possibility of a social e-book by analyzing reader experiences,0,en,50,57,10.1007/978-3-642-39371-6_6,null,2013,null,null
53e997e4b7602d9701fdcbe4,53a72ea220f7420be8c9573e,5f71b5d91c455f439fe53c8a,"List(53f46944dabfaefedbb94054, 53f438a2dabfaee2a1cfba45, 53f4d24adabfaeedcf780064, 5430219cdabfaeca69bcbdab, 5484ab76dabfaed7b5fa1b09)","List(53e9b715b7602d97042ab658, 53e9a1dbb7602d9702adac6a, 53e9b56cb7602d97040a4772, 53e99d88b7602d97026469fb, 53e9b0a5b7602d9703b1344c, 53e9b0c2b7602d9703b32bfc, 53e9b648b7602d97041a30bb, 53e99976b7602d97021b6824, 53e9a533b7602d9702e5845d, 53e99a04b7602d970224e686)","List(af-agentspeak agent programming language, hybrid control architecture, multi-agent programming, platform service, agent code, university college dublin, previous year, agent factory platform)","List(Computer software, Architecture, Software engineering, Factory, Computer science, CONTEST, Multi-agent system, Artificial intelligence)",Bogtrotters in space,2,en,197,207,10.1007/978-3-642-31915-0_12,null,2011,null,null
53e997e8b7602d9701fdfeec,555036b57cea80f95414abdc,17179881764,"List(53f4475bdabfaeecd69b037a, 53f44082dabfaee43ec6cfb0, 53f3af18dabfae4b34b108b0, 5447fcdcdabfae87b7dbc44b, 53f4627adabfaefedbb798dc, 5601a96d45cedb3395e88e81, 53f42ef3dabfaee43ebd8d3b)","List(53e99adcb7602d970235febb, 53e9ade9b7602d97037f3d3f, 53e9984bb7602d9702079d8f, 53e9a2ddb7602d9702be93df, 56d833a5dabfae2eee3216f7)","List(convex position, delaunay triangulation, n black point, delaunay graph, set b, proximity graphs, graph drawing, blocking delaunay triangulations, witness graphs, n-1 white point, following bound, general position, black point, white point, smallest set)","List(Graph drawing, General position, Combinatorics, Convex position, Mathematics, Delaunay triangulation, Delaunay graph)",Blocking Delaunay triangulations.,16,en,154,159,10.1016/j.comgeo.2012.02.005,null,2013,46,2
53e997e9b7602d9701fe40fc,53e18bf620f7dfbc07e8feb2,5f71b2bf1c455f439fe3df7b,"List(53f437f0dabfaee0d9b7327a, 53f43754dabfaee0d9b6d422)","List(53e9a4fab7602d9702e1b383, 56d8d01ddabfae2eee9cdb77, 53e99f8cb7602d970285f956, 53e9ba84b7602d97046a8022)","List(previously-existing style, new work, piet mondrian, particular artistic style, novel artistic style, case-based reasoning, new artwork, case-based art, interesting artwork, reasoning method, dutch painter, artificial intelligent, case base reasoning)","List(Evolutionary algorithm, Creative design, Computer science, Painting, Artificial intelligence, Case-based reasoning, Mondrian)",Case-Based Art,0,en,237,251,10.

In [0]:
dblp_df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Venue: string (nullable = true)
 |-- Org: string (nullable = true)
 |-- Authors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- References: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- FOS: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Title: string (nullable = true)
 |-- NoCitations: integer (nullable = true)
 |-- Lang: string (nullable = true)
 |-- PageStart: integer (nullable = true)
 |-- PageEnd: integer (nullable = true)
 |-- DOI: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Issue: integer (nullable = true)



### 4. Load DFs as Delta tables
TODO: create orgs delta table

In [0]:
# DBLP fact table
dblp_df.write.format('delta').mode('overwrite').saveAsTable('dblp_fact_table')

In [0]:
# Venue table
venues_df.write.format("delta").mode("overwrite").saveAsTable("venues") 

In [0]:
# Author table
authors_df.write.format('delta').mode('overwrite').saveAsTable('authors')

In [0]:
# Organization


#### 5. Adding operation

#### 6. Queries